# Experiments

#### Imports:

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


from tensorflow import keras
from sklearn import preprocessing

#### Aux functions:

In [2]:
def split_data(train_x, train_y, training=0.8, validation=0.5):
    train_size = training

    train_cnt = math.floor(train_x.shape[0] * train_size)
    x_train = train_x.iloc[0:train_cnt].values
    y_train = train_y.iloc[0:train_cnt].values
    x_test = train_x.iloc[train_cnt:]
    y_test = train_y.iloc[train_cnt:]

    division = validation

    train_cnt = math.floor(x_test.shape[0] * division)
    x_validate = x_test.iloc[0:train_cnt].values
    y_validate = y_test.iloc[0:train_cnt].values
    x_test = x_test.iloc[train_cnt:].values
    y_test = y_test.iloc[train_cnt:].values

    #Preprocess the data
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_validate = min_max_scaler.fit_transform(x_validate)
    #x_train = min_max_scaler.fit_transform(x_train)
    #x_test = min_max_scaler.fit_transform(x_test)
    
    return x_train, y_train, x_test, y_test, x_validate, y_validate
    

#### Load data:

In [3]:
frame = pd.read_csv("complete-frame.csv", sep=",")
"""
train_x = pd.concat([frame.no_mutations, frame.line_coverage, frame.isAssertionRoulette, frame.isEagerTest, frame.isLazyTest,
frame.isMysteryGuest, frame.isSensitiveEquality, frame.isResourceOptimism, frame.isForTestersOnly,
frame.isIndirectTesting, frame.LOC_prod, frame.HALSTEAD_prod, frame.RFC_prod, frame.CBO_prod, frame.MPC_prod, frame.IFC_prod, frame.DAC_prod,frame.DAC2_prod, frame.LCOM1_prod, frame.LCOM2_prod,
frame.LCOM3_prod, frame.LCOM4_prod, frame.CONNECTIVITY_prod, frame.LCOM5_prod, frame.COH_prod, frame.TCC_prod,
frame.LCC_prod, frame.ICH_prod, frame.WMC_prod, frame.NOA_prod, frame.NOPA_prod, frame.NOP_prod,
frame.McCABE_prod, frame.BUSWEIMER_prod, frame.LOC_test, frame.HALSTEAD_test, frame.RFC_test, frame.CBO_test,
frame.MPC_test, frame.IFC_test, frame.DAC_test, frame.DAC2_test, frame.LCOM1_test, frame.LCOM2_test,
frame.LCOM3_test, frame.LCOM4_test, frame.CONNECTIVITY_test, frame.LCOM5_test, frame.COH_test, frame.TCC_test,
frame.LCC_test, frame.ICH_test, frame.WMC_test, frame.NOA_test, frame.NOPA_test, frame.NOP_test, frame.McCABE_test,
frame.BUSWEIMER_test, frame.csm_CDSBP, frame.csm_CC, frame.csm_FD, frame.csm_Blob, frame.csm_SC, frame.csm_MC,
frame.csm_LM, frame.csm_FE, frame.prod_readability, frame.test_readability], axis=1).round(2)

"""

data_x = pd.concat([frame.no_mutations,
                     frame.line_coverage,
                     #frame.isEagerTest,
                     frame.LOC_prod, frame.LOC_test, frame.WMC_prod,
                     frame.LCOM1_prod, frame.LCOM2_prod,
                     frame.LCOM4_prod, frame.McCABE_prod,
                     frame.RFC_prod, frame.MPC_prod,
                     frame.RFC_test, frame.MPC_test,
                     frame.LCOM1_test, frame.LCOM2_test,
                     frame.LCOM4_test, frame.LCC_test,
                     frame.LCC_test, frame.WMC_test,
                     frame.McCABE_test, frame.NOP_prod,
                     frame.NOA_prod], axis = 1).round(2)

data_y = pd.concat([frame.mutation], axis = 1)

#labels = [1,2,3,4,5]
#bins = [0,2,4,6,8,10]

#frame['mutation_bins'] = pd.cut(frame.mutation.round(1).mul(10), bins=bins, labels = labels, include_lowest=True)
#frame['mutation_bins'] = pd.cut(frame.mutation, bins=bins, labels = labels, include_lowest=True)
#print(frame.mutation_bins)

#train_y = pd.concat([frame.mutation_bins], axis = 1)

## Experiment #1: Effective Vs. Non Effective
    - 2 classes: effective tests and non-effective tests:
        - Effective > median(mutation_score)
        - Non Effective < media(mutation_score)

In [6]:
#Edit the y axis data according to the description:
labels = [1,2]
bins = [0,frame.mutation.median(),1]
frame['mutation_bins'] = pd.cut(frame.mutation, bins=bins, labels = labels, include_lowest=True)
data_y = pd.concat([frame.mutation_bins], axis = 1)

x_train, y_train, x_test, y_test, x_validate, y_validate = split_data(data_x, data_y)
print(x_train)


# Define the model architecture

model = keras.Sequential()
model.add(keras.layers.Dense(22, input_dim=22, activation=tf.nn.relu))
model.add(keras.layers.Dense(18, activation=tf.nn.relu))
model.add(keras.layers.Dense(12, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping_monitor = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)


model.fit(x_train, y_train, epochs=1000, verbose=1,
          validation_data=(x_validate, y_validate),
          callbacks=[early_stopping_monitor])

# Evaluate the model using test dataset.
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)

#Check Overfeet
#test_loss, test_acc = model.evaluate(x_train, y_train)


[[ 36.     0.93  95.   ...   6.     0.     0.  ]
 [ 28.     0.67 271.   ...  30.     0.     0.  ]
 [ 74.     0.97 375.   ...  24.     0.     0.  ]
 ...
 [  4.     1.    95.   ...   8.     0.     2.  ]
 [  4.     1.    95.   ...  21.     0.     2.  ]
 [  6.     1.    87.   ...   7.     0.     1.  ]]
Train on 1928 samples, validate on 241 samples
Epoch 1/1000
1928/1928 [==============================] - 1s 591us/sample - loss: 22.6130 - accuracy: 0.2505 - val_loss: 3.7057 - val_accuracy: 0.2407
Epoch 2/1000
1928/1928 [==============================] - 0s 97us/sample - loss: 1.8247 - accuracy: 0.4388 - val_loss: 1.1223 - val_accuracy: 0.5394
Epoch 3/1000
1928/1928 [==============================] - 0s 91us/sample - loss: 0.9577 - accuracy: 0.5176 - val_loss: 0.8925 - val_accuracy: 0.5353
Epoch 4/1000
1928/1928 [==============================] - 0s 90us/sample - loss: 0.8026 - accuracy: 0.5348 - val_loss: 0.5693 - val_accuracy: 0.7884
Epoch 5/1000
1928/1928 [==============================]

Test accuracy: 0.7355372


## Experiment #1: Results

- 0s 54us/sample - loss: 0.7057 - accuracy: 0.7355 Test accuracy: **0.7355372**


## Experiment #2: Predicting Mutation Score
    - 11 categories of mutation score: from 0, 0.1, 0.2, .... 1
    - The goal: predict the mutation score

In [7]:
data_y = pd.concat([frame.mutation], axis = 1).round(1).mul(10)

x_train, y_train, x_test, y_test, x_validate, y_validate = split_data(data_x, data_y)

# Define the model architecture

model = keras.Sequential()
model.add(keras.layers.Dense(22, input_dim=22, activation=tf.nn.relu))
model.add(keras.layers.Dense(18, activation=tf.nn.relu))
model.add(keras.layers.Dense(12, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.relu))
model.add(keras.layers.Dense(11, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping_monitor = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)


model.fit(x_train, y_train, epochs=1000, verbose=1,
          validation_data=(x_validate, y_validate),
          callbacks=[early_stopping_monitor])

# Evaluate the model using test dataset.
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)

Train on 1928 samples, validate on 241 samples
Epoch 1/1000
1928/1928 [==============================] - 1s 619us/sample - loss: 41.2130 - accuracy: 0.1151 - val_loss: 7.8926 - val_accuracy: 0.0747
Epoch 2/1000
1928/1928 [==============================] - 0s 93us/sample - loss: 7.9864 - accuracy: 0.1068 - val_loss: 3.3282 - val_accuracy: 0.1452
Epoch 3/1000
1928/1928 [==============================] - 0s 94us/sample - loss: 4.8984 - accuracy: 0.1255 - val_loss: 2.7534 - val_accuracy: 0.1162
Epoch 4/1000
1928/1928 [==============================] - 0s 90us/sample - loss: 3.5364 - accuracy: 0.1478 - val_loss: 2.4270 - val_accuracy: 0.1535
Epoch 5/1000
1928/1928 [==============================] - 0s 90us/sample - loss: 3.3432 - accuracy: 0.1494 - val_loss: 2.3610 - val_accuracy: 0.1535
Epoch 6/1000
1928/1928 [==============================] - 0s 89us/sample - loss: 2.8419 - accuracy: 0.1592 - val_loss: 2.3635 - val_accuracy: 0.1618
Epoch 7/1000
1928/1928 [==============================] -

1928/1928 [==============================] - 0s 100us/sample - loss: 2.2792 - accuracy: 0.1566 - val_loss: 2.1866 - val_accuracy: 0.1701
Epoch 56/1000
1928/1928 [==============================] - 0s 89us/sample - loss: 2.2790 - accuracy: 0.1598 - val_loss: 2.1733 - val_accuracy: 0.1618
Epoch 57/1000
1928/1928 [==============================] - 0s 90us/sample - loss: 2.2789 - accuracy: 0.1608 - val_loss: 2.1727 - val_accuracy: 0.1660
Epoch 58/1000
1928/1928 [==============================] - 0s 92us/sample - loss: 2.2777 - accuracy: 0.1608 - val_loss: 2.1589 - val_accuracy: 0.1701
Epoch 59/1000
1928/1928 [==============================] - 0s 92us/sample - loss: 2.2798 - accuracy: 0.1618 - val_loss: 2.1480 - val_accuracy: 0.1701
Epoch 60/1000
1928/1928 [==============================] - 0s 92us/sample - loss: 2.2776 - accuracy: 0.1603 - val_loss: 2.1679 - val_accuracy: 0.1618
Epoch 61/1000
1928/1928 [==============================] - 0s 96us/sample - loss: 2.4821 - accuracy: 0.1603 - val

1928/1928 [==============================] - 0s 100us/sample - loss: 2.2971 - accuracy: 0.1608 - val_loss: 2.4221 - val_accuracy: 0.1701
Epoch 110/1000
1928/1928 [==============================] - 0s 97us/sample - loss: 2.2886 - accuracy: 0.1551 - val_loss: 2.4029 - val_accuracy: 0.1950
Epoch 111/1000
1928/1928 [==============================] - 0s 95us/sample - loss: 2.2836 - accuracy: 0.1618 - val_loss: 2.2382 - val_accuracy: 0.2656
Epoch 112/1000
1928/1928 [==============================] - 0s 92us/sample - loss: 2.2460 - accuracy: 0.1758 - val_loss: 2.0236 - val_accuracy: 0.3693
Epoch 113/1000
1928/1928 [==============================] - 0s 93us/sample - loss: 2.2343 - accuracy: 0.1795 - val_loss: 1.9994 - val_accuracy: 0.3817
Epoch 114/1000
1928/1928 [==============================] - 0s 91us/sample - loss: 2.2286 - accuracy: 0.1831 - val_loss: 1.9316 - val_accuracy: 0.4108
Epoch 115/1000
1928/1928 [==============================] - 0s 93us/sample - loss: 2.2459 - accuracy: 0.1763

Test accuracy: 0.23966943


### Best Result: 
0s 56us/sample - loss: 2.1090 - accuracy: 0.2397 Test accuracy: **0.23966943**

(Not an over fit: accuracy on trainin data is 0.182)

In [8]:
#Check Overfeet
test_loss, test_acc = model.evaluate(x_train, y_train)

1928/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================